In [1]:
from torch.utils.data import DataLoader,Dataset
import torch
import pandas 


In [6]:
delay = list(range(10))
loss = list(range(10,20))

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [15]:
dataframe = pd.DataFrame({'delay':delay,'loss':loss})

#将DataFrame存储为csv,index表示是否显示行名，default=True
dataframe.to_csv("test.csv",index=False,sep=',')

In [17]:
data = pd.read_csv("test.csv")
 

In [32]:
package = ('1'*1500).encode()

In [35]:
len(package)

1500

In [6]:
import pandas as pd
train_df = pd.read_csv('./dataset/train.csv')
raw_data = train_df[train_df['building_id'].apply(lambda x:x==1)&
             train_df['meter_reading'].apply(lambda x:x>0)&
             train_df['meter'].apply(lambda x:x==0)]

In [7]:
class RTTData(Dataset):
    def __init__(self, root_dir, history_len=10, forecast_len=1, interval=5, train=True):
    
        self.history_len = history_len
        self.forecast_len = forecast_len
        self.interval = interval
        train_df = pandas.read_csv(root_dir+'server.csv')
        self.raw_data = train_df.values[:,1]
       
        split_index = int(len(self.raw_data)*0.7)
#         print('split_index is ',split_index)
        if(train==True):
            self.raw_data = self.raw_data[200:split_index].astype('float32')
#             print('trainset raw_data size: ', len(self.raw_data))
        else:
            self.raw_data = self.raw_data[split_index:].astype('float32')
#             print('testset raw_data size: ', len(self.raw_data))
        self.len = int((self.raw_data.shape[0]-self.history_len-self.forecast_len)/self.interval)+1
    
    def __getitem__(self, index):
        if(index<self.len):
            x_start = self.interval*index
            y_start = x_start+self.history_len
            x_data = torch.from_numpy(self.raw_data[x_start:x_start+self.history_len])
            y_reg = torch.from_numpy(self.raw_data[y_start:y_start+self.forecast_len]) 
            
            y_future = self.raw_data[y_start:y_start+5]
            if(y_future[0]>x_data[-1] and y_future[1]>y_future[0] and y_future[2]>y_future[1]):
            #if(y_future[0]>x_data[-1] and y_future[1]>x_data[-1] and y_future[2]>x_data[-1]):
                y_cla = torch.tensor(1)
            else:
                y_cla = torch.tensor(0)
            return x_data, (y_reg, y_cla)
        else:
            return None
    
    def __len__(self):
        return self.len
    

In [8]:
dataset = RTTData('dataset/')

In [12]:
print(dataset[2])

(tensor([871.3520, 686.8860, 547.6130, 563.1110, 539.9760, 362.8690, 376.5720,
        255.3820, 238.1110, 219.2210]), (tensor([225.3110]), tensor(0)))
